In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
import warnings

In [3]:
warnings.filterwarnings('ignore')

In [4]:
batch_size = 64
epochs = 100
latent_dim = 256 #latent dimensionality of the encoding space
num_samples = 10000 #Number of samples to train on
data_path = 'fra-eng/fra.txt'

In [5]:
input_texts = []
target_texts = []
#vectorize the data
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines=f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')[:2]  # Ensure correct splitting
    # We use tab as the "start sequence" character for targets, and '\n' as "end sequence" character
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [6]:
len(input_texts)

10000

In [7]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [8]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 91
Max sequence length for inputs: 14
Max sequence length for outputs: 59


In [9]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)]
)
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)]
)

In [10]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69}

In [11]:
# One-hot encoding using numpy
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), 
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), 
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), 
    dtype='float32')

In [12]:
for i, (input_text, target_text) in enumerate(zip(input_texts,target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]] = 1.
    encoder_input_data[i, t, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        #decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t>0:
            #decoder_target_data will be ahead by one timestep
            #and will not include the start character.
            decoder_target_data[i, t-1, target_token_index[char]] =1.
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [13]:
encoder_input_data[0].shape

(14, 70)

In [14]:
#define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#We discard 'encoder outputs' and only keep the states.
encoder_states = [state_h, state_c]

In [15]:
#Set up the decoder, using 'encoder_states' as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))
#We set up our decoder to return full output sequences,
#and to return internal states as well, We dont use the
#return states in the training model, but we will use them in inference
decoder_lstm= LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense= Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [18]:
#Defining model that will turn encoder_input_data & decoder_input_datea
#into decoder_target_data

model= Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics= ['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs= epochs, validation_split=0.2)

Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 38s 278ms/step - accuracy: 0.9542 - loss: 0.1504 - val_accuracy: 0.8757 - val_loss: 0.5131
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 35s 282ms/step - accuracy: 0.9566 - loss: 0.1436 - val_accuracy: 0.8767 - val_loss: 0.5167
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 35s 283ms/step - accuracy: 0.9569 - loss: 0.1423 - val_accuracy: 0.8758 - val_loss: 0.5183
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 35s 283ms/step - accuracy: 0.9576 - loss: 0.1410 - val_accuracy: 0.8758 - val_loss: 0.5193
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 36s 287ms/step - accuracy: 0.9581 - loss: 0.1382 - val_accuracy: 0.8759 - val_loss: 0.5218
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 36s 284ms/step - accuracy: 0.9580 - loss: 0.1381 - val_accuracy: 0.8765 - val_loss: 0.5285
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 36s 285ms/step - accuracy: 0.9588 - loss: 0.1364 - val_accuracy: 0.8757 - val_loss: 0.5281
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 37s 293ms/step - accuracy: 0.9597 -

In [19]:
model.save("s2s_model.keras")

Run inference (sampling)
encode input and retrieve initial decoder state
run one step of decoder with this initial state and a "start of sequence" token as target. Output will be the next target token.
Repeat with the current target token and current states

In [21]:
import tensorflow as tf

In [22]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = tf.keras.models.load_model("s2s_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = tf.keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = tf.keras.Input(shape=(latent_dim,))
decoder_state_input_c = tf.keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = tf.keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [23]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: En route !

-
Input sentence: Go.
Decoded sentence: En route !

-
Input sentence: Go.
Decoded sentence: En route !

-
Input sentence: Go.
Decoded sentence: En route !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run!
Decoded sentence: Fuyons !

-
Input sentence: Run.
Decoded sentence: Filez !

-
Input sentence: Run.
Decoded sentence: Filez !

-
Input sentence: Run.
Decoded sentence: Filez !

-
Input sentence: Run.
Decoded sentence: Filez !

-
Input sentence: Run.
Decoded sentence: Filez !

-
Input sentence: Run.
Decoded sente